In [1]:
using CSV
using CategoricalArrays
using DataFrames
using HypothesisTests
using LinearAlgebra
using LogExpFunctions
using MixedModels
using SpecialFunctions
using Statistics

In [2]:
# load the EM library

full = false    # Maintain full covariance matrix (vs a diagional one) at the group level
emtol = 1e-2    # stopping condition (relative change) for EM

em_directory = "" #directory where EM functions are stored, from Daw lab.

savepath = "" #location to save cross-validated likelihood data from each rat and session

push!(LOAD_PATH,em_directory)
using EM

#load the likelihood functions
lik_directory = "" #directory where likelihood functions are stored, from modules folder.

push!(LOAD_PATH,lik_directory)
using likFuncs

In [3]:
# load the data

#insert df path into function
data = CSV.read("", DataFrame);

# get rid of frames with no location
data = data[data.pairedHexState .>= 0,:];

# get rid of sessions with no ramp
data = data[data.rat .!= "IM-1292",:];
data = data[data.session .!= 90,:];
data = data[data.session .!= 92,:];
data = data[data.session .!= 94,:];

data.session = CategoricalArray(data.session);

### fit TD lambda 

In [ ]:
data.sub = data.session
bsTDlambdamodel = []
lsTDlambdamodel = []
isTDlambdamodel= []
betaErrors = []
betaPvals = []

for rat in levels(data.rat)
    datarat = data[data.rat .== rat,:]
    subs = levels(datarat.sub)
    NS = length(subs)

    X = ones(NS)
    startbetas = [0 0 0 0 0 0 0 0.]
    startsigma = [1, 1, 1, 1, 1, 1, 1, 1]


    (betas,sigma,x,l,h) = em(datarat,subs,X,startbetas,startsigma,tdlambdalik; emtol=emtol, full=full,maxiter=1000);

    bsTDlambdamodel = [bsTDlambdamodel; vec(betas)]
    lsTDlambdamodel = [lsTDlambdamodel; lml(x,l,h)]
    isTDlambdamodel = [isTDlambdamodel; iaic(x,l,h,betas,sigma)]
    loocvTDlambdamodel = [loocvTDlambdamodel; loocv(datarat,subs,x,X,betas,sigma,tdlambdalik;emtol=emtol, full=full)]
    loocvDfTDlambdamodel = DataFrame(rat=repeat([rat],length(loocvTDlambdamodel)),loocv=loocvTDlambdamodel)
    CSV.write(savepath*"loocvTdLambdaModel_"*rat*".csv",loocvDfTDlambdamodel)
end

## fit 2 component model 

In [ ]:
data.sub = data.session
bs2Vmodel = []
ls2Vmodel = []
is2Vmodel= []
betaErrors = []
betaPvals = []

for rat in levels(data.rat)
    datarat = data[data.rat .== rat,:]
    subs = unique(datarat.sub)
    NS = length(subs)

    X = ones(NS)
    startbetas = [0 0 0 0 0 0 0 0.]
    startsigma = [1, 1, 1, 1, 1, 1, 1, 1]


    (betas,sigma,x,l,h) = em(datarat,subs,X,startbetas,startsigma,etliksep; emtol=emtol, full=full,maxiter=1000);

    bs2Vmodel = [bs2Vmodel; vec(betas)]
    ls2Vmodel = [ls2Vmodel; lml(x,l,h)]
    is2Vmodel = [is2Vmodel; iaic(x,l,h,betas,sigma)]
    loocv2Vmodel = loocv(datarat,subs,x,X,betas,sigma,etliksep;emtol=emtol, full=full)
    loocvDf2Vmodel = DataFrame(rat=repeat([rat],length(loocv2Vmodel)),loocv=loocv2Vmodel)
    CSV.write(savepath*rat*"loocvDualModel_"*".csv",loocvDf2Vmodel)
    #also keep value-betas for each rat and session, for significance analysis
    (standarderrors,pvalues,covmtx) = emerrors(datarat,subs,x,X,h,betas,sigma,etliksep)
    betaErrors = [betaErrors; standarderrors]
    betaPvals = [betaPvals; pvalues]
end

# fit two nested versions of the dual-component model

### local only

In [ ]:
data.sub = data.session
bsLocalmodel = []
lsLocalmodel = []
isLocalmodel= []

for rat in levels(data.rat)
    datarat = data[data.rat .== rat,:]
    subs = unique(datarat.sub)
    NS = length(subs)

    X = ones(NS)
    startbetas = [0 0 0 0 0.]
    startsigma = [1, 1, 1, 1, 1]


    (betas,sigma,x,l,h) = em(datarat,subs,X,startbetas,startsigma,etliksepNoTD1MB; emtol=emtol, full=full,maxiter=1000);

    bsLocalmodel = [bsLocalmodel; vec(betas)]
    lsLocalmodel = [lsLocalmodel; lml(x,l,h)]
    isLocalmodel = [isLocalmodel; iaic(x,l,h,betas,sigma)]
    loocvLocalmodel = loocv(datarat,subs,x,X,betas,sigma,etliksepNoTD1MB;emtol=emtol, full=full)
    loocvDfLocalmodel = DataFrame(rat=repeat([rat],length(loocvLocalmodel)),loocv=loocvLocalmodel)
    CSV.write(savepath*rat*"loocvLocalModel_"*".csv",loocvLocalmodel)
end

### global only

In [ ]:
data.sub = data.session
bsGlobalmodel = []
lsGlobalmodel = []
isGlobalmodel= []

for rat in levels(data.rat)
    datarat = data[data.rat .== rat,:]
    subs = unique(datarat.sub)
    NS = length(subs)

    X = ones(NS)
    startbetas = [0 0 0 0 0 0.]
    startsigma = [1, 1, 1, 1, 1, 1]


    (betas,sigma,x,l,h) = em(datarat,subs,X,startbetas,startsigma,etliksepNoMF; emtol=emtol, full=full,maxiter=1000);

    bsGlobalmodel = [bsGlobalmodel; vec(betas)]
    lsGlobalmodel = [lsGlobalmodel; lml(x,l,h)]
    isGlobalmodel = [isGlobalmodel; iaic(x,l,h,betas,sigma)]
    loocvGlobalmodel = loocv(datarat,subs,x,X,betas,sigma,etliksepNoMF;emtol=emtol, full=full)
    loocvDfGlobalmodel = DataFrame(rat=repeat([rat],length(loocvGlobalmodel)),loocv=loocvGlobalmodel)
    CSV.write(savepath*rat*"loocvGlobalModel_"*".csv",loocvGlobalmodel)
end